In [1]:
!pip install -q --upgrade pymupdf
!pip install -q ultralytics
!pip install -q tools



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import json
import random
import shutil

# Make sure you have the correct libs:
# !pip uninstall -y fitz
# !pip install --upgrade pymupdf ultralytics

import fitz  # from pymupdf
from ultralytics import YOLO

# ----------------- INPUT PATHS -----------------
files_path = "/content/drive/MyDrive/HACKATHON_ARMETA/selected_output/pdfs"
output_json_path = "/content/drive/MyDrive/HACKATHON_ARMETA/selected_output/selected_annotations.json"

# Where to build YOLO dataset
dataset_root = "/content/drive/MyDrive/HACKATHON_ARMETA/selected_output/yolo_format"

model_path = "/content/drive/MyDrive/HACKATHON_ARMETA/trained_models/nano_50epochs.pt"

# ----------------- CLASS MAPPING -----------------
category2id = {
    "signature": 0,
    "stamp": 1,
    "qr": 2,
}


def build_yolo_dataset_from_pdfs(
    pdf_dir,
    json_path,
    dataset_root,
    train_ratio=0.8,
    seed=42,
):
    """
    1) Reads JSON annotations.
    2) Renders annotated PDF pages into images with correct size.
    3) Writes YOLOv8 txt labels.
    4) Splits into train/val and creates dataset.yaml.

    Returns: path to dataset.yaml
    """

    # Staging dirs for all images/labels
    images_all = os.path.join(dataset_root, "images_all")
    labels_all = os.path.join(dataset_root, "labels_all")
    os.makedirs(images_all, exist_ok=True)
    os.makedirs(labels_all, exist_ok=True)

    # Load JSON
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    image_label_pairs = []  # list of (image_path, label_path)

    for pdf_name, pages in data.items():
        pdf_path = os.path.join(pdf_dir, pdf_name)
        if not os.path.exists(pdf_path):
            print(f"WARNING: PDF not found: {pdf_path}, skipping.")
            continue

        try:
            doc = fitz.open(pdf_path)
        except Exception as e:
            print(f"ERROR opening {pdf_path}: {e}")
            continue

        pdf_stem, _ = os.path.splitext(pdf_name)

        for page_key, page_data in pages.items():
            # page_key like "page_3"
            try:
                page_num = int(page_key.split("_")[1])  # 1-based
            except Exception:
                print(f"WARNING: cannot parse page number from {page_key} in {pdf_name}, skipping page.")
                continue

            page_index = page_num - 1  # PyMuPDF is 0-based
            if page_index < 0 or page_index >= len(doc):
                print(f"WARNING: page index {page_index} out of range for {pdf_name}, skipping.")
                continue

            page = doc[page_index]

            # Target size from JSON
            target_w = page_data["page_size"]["width"]
            target_h = page_data["page_size"]["height"]

            # Compute scaling so that rendered image matches page_size
            rect = page.rect  # in PDF points
            scale_x = target_w / rect.width
            scale_y = target_h / rect.height
            matrix = fitz.Matrix(scale_x, scale_y)

            pix = page.get_pixmap(matrix=matrix)
            # Sanity: pix.width, pix.height should be ~target_w, target_h
            # Save image
            img_basename = f"{pdf_stem}_page_{page_num}.png"
            img_path = os.path.join(images_all, img_basename)
            pix.save(img_path)

            # Build YOLO label
            label_path = os.path.join(labels_all, f"{pdf_stem}_page_{page_num}.txt")
            label_lines = []

            for ann in page_data.get("annotations", []):
                # ann is like {"annotation_117": {...}}
                inner = next(iter(ann.values()))
                category = inner["category"]

                if category not in category2id:
                    print(f"WARNING: unknown category '{category}', skipping.")
                    continue

                cls_id = category2id[category]
                bbox = inner["bbox"]

                x = float(bbox["x"])
                y = float(bbox["y"])
                w = float(bbox["width"])
                h = float(bbox["height"])

                # Convert from top-left (x,y,w,h) in pixels to normalized YOLO (cx, cy, w, h)
                cx = x + w / 2.0
                cy = y + h / 2.0

                cx_n = cx / target_w
                cy_n = cy / target_h
                w_n = w / target_w
                h_n = h / target_h

                label_lines.append(
                    f"{cls_id} {cx_n:.6f} {cy_n:.6f} {w_n:.6f} {h_n:.6f}"
                )

            if not label_lines:
                # No objects on this page → either skip or create empty file.
                # Here we SKIP entirely (no negative examples from this pipeline).
                continue

            with open(label_path, "w", encoding="utf-8") as lf:
                lf.write("\n".join(label_lines))

            image_label_pairs.append((img_path, label_path))

        doc.close()

    print(f"Total annotated pages converted: {len(image_label_pairs)}")

    # ----------------- TRAIN / VAL SPLIT -----------------
    random.seed(seed)
    random.shuffle(image_label_pairs)

    n_total = len(image_label_pairs)
    n_train = int(n_total * train_ratio)
    train_pairs = image_label_pairs[:n_train]
    val_pairs = image_label_pairs[n_train:]

    # Final YOLO dirs
    train_img_dir = os.path.join(dataset_root, "images/train")
    val_img_dir = os.path.join(dataset_root, "images/val")
    train_lbl_dir = os.path.join(dataset_root, "labels/train")
    val_lbl_dir = os.path.join(dataset_root, "labels/val")
    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)

    def move_pair(src_img, src_lbl, dst_img_dir, dst_lbl_dir):
        img_name = os.path.basename(src_img)
        lbl_name = os.path.basename(src_lbl)
        shutil.copy2(src_img, os.path.join(dst_img_dir, img_name))
        shutil.copy2(src_lbl, os.path.join(dst_lbl_dir, lbl_name))

    for img_path, lbl_path in train_pairs:
        move_pair(img_path, lbl_path, train_img_dir, train_lbl_dir)
    for img_path, lbl_path in val_pairs:
        move_pair(img_path, lbl_path, val_img_dir, val_lbl_dir)

    print(f"Train pages: {len(train_pairs)}, Val pages: {len(val_pairs)}")

    # ----------------- CREATE dataset.yaml -----------------
    dataset_yaml_path = os.path.join(dataset_root, "dataset.yaml")
    yaml_text = f"""path: {dataset_root}

train: images/train
val: images/val

names:
  0: signature
  1: stamp
  2: qr
"""
    with open(dataset_yaml_path, "w", encoding="utf-8") as f:
        f.write(yaml_text)

    print(f"dataset.yaml written to: {dataset_yaml_path}")
    return dataset_yaml_path


def train_yolov8n_on_dataset(dataset_yaml_path):
    """
    Train YOLOv8n on the generated dataset.
    Settings tuned for a small dataset.
    """
    model = YOLO(model_path)  # COCO-pretrained nano

    model.train(
        data=dataset_yaml_path,
        epochs=200,        # small dataset -> more epochs
        patience=40,       # early stopping
        imgsz=1024,        # adapt if needed
        batch=-1,          # let YOLO auto-tune batch size
        optimizer="AdamW",
        lr0=0.001,
        lrf=0.01,
        cos_lr=True,
        device=0,          # GPU 0; use 'cpu' if no GPU
        verbose=True,
    )


def build_and_train():
    dataset_yaml = build_yolo_dataset_from_pdfs(
        pdf_dir=files_path,
        json_path=output_json_path,
        dataset_root=dataset_root,
        train_ratio=0.8,
        seed=42,
    )
    train_yolov8n_on_dataset(dataset_yaml)


# Run everything
build_and_train()


Mounted at /content/drive
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Total annotated pages converted: 91
Train pages: 72, Val pages: 19
dataset.yaml written to: /content/drive/MyDrive/HACKATHON_ARMETA/selected_output/yolo_format/dataset.yaml
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/HACKATHON_ARMETA/selected_output/yolo_format/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, 